# <div align="center"><b> Vit Base </b></div>

<div align="right">

<!-- [![Binder](http://mybinder.org/badge.svg)](https://mybinder.org/) -->
[![nbviewer](https://img.shields.io/badge/render-nbviewer-orange?logo=Jupyter)](https://nbviewer.org/github/brunomaso1/vision-transformer/blob/main/notebooks/3.03-vit-base.ipynb)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/brunomaso1/vision-transformer/blob/main/notebooks/3.03-vit-base.ipynb)

</div>

* * *

<style>
/* Limitar la altura de las celdas de salida en html */
.jp-OutputArea.jp-Cell-outputArea {
    max-height: 500px;
}
</style>

🛻 <em><font color='MediumSeaGreen'>  Instalaciones: </font></em> 🛻


Este notebook utiliza [Poetry](https://python-poetry.org/) para la gestión de dependencias.
Primero instala Poetry siguiendo las instrucciones de su [documentación oficial](https://python-poetry.org/docs/#installation).
Luego ejecuta el siguiente comando para instalar las dependencias necesarias y activar el entorno virtual:

- Bash:

```bash
poetry install
eval $(poetry env activate)
```

- PowerShell:

```powershell
poetry install
Invoke-Expression (poetry env activate)
```

> 📝 <em><font color='Gray'>Nota:</font></em> Para agregar `pytorch` utilizando Poetry, se utiliza el siguiente comando:
> ```bash
> # Más info: https://github.com/python-poetry/poetry/issues/6409
> potery source add --priority explicit pytorch_gpu https://download.pytorch.org/whl/cu128 # Seleccionar la wheel adecuada para tu GPU
> poetry add --source pytorch_gpu torch torchvision 
> ```

✋ <em><font color='DodgerBlue'>Importaciones:</font></em> ✋

In [18]:
# Recarga automática de módulos en Jupyter Notebook
%reload_ext autoreload
%autoreload 2

# Importación de bibliotecas necesarias
import random, requests, os

# Configuración de logging
from loguru import logger

# Pandas y visualización
import pandas as pd

from PIL import Image

# PyTorch
import torch
from torch.utils.data import DataLoader
import torchvision.transforms.functional as TF

# NumPy y utilidades
import numpy as np
from functools import partial

# Evaluación
import evaluate

# Modelos y procesamiento de imágenes (Transformers)
from transformers import (
    AutoImageProcessor,
    AutoModelForImageClassification,
    DefaultDataCollator,
    ViTForImageClassification,
    ViTImageProcessor,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback,
    # Swinv2ForImageClassification,  # Descomentar si se usa explícitamente
)
from transformers.integrations import MLflowCallback


# Modulos propios
# Módulos propios
from vision_transformer.plots import show_image_grid, plot_confusion_matrix, plot_metric
from vision_transformer.dataset import load_huggingface_dataset
from vision_transformer.features import VitBaseTransforms
from vision_transformer.utils import MulticlassAccuracy
from vision_transformer.config import (
    RANDOM_SEED,
    MODEL_NAME_VIT_BASE,
    MODELS_DIR_VIT_BASE,
    MLFLOW_URL,
    PREFECT_URL,
    MODELS_DIR,
    DATASET_NAME,
    DATASET_VERSION,
    HISTORY_FILENAME,
    PREDICTIONS_FILENAME,
    FIGURES_DIR,
    METRICS_FILENAME
)

import mlflow

🔧 <em><font color='tomato'>Configuraciones:</font></em> 🔧


In [2]:
random.seed(RANDOM_SEED)

# Checkpoints a utilizar
MODEL_NAME = MODEL_NAME_VIT_BASE
MODEL_FOLDER = MODELS_DIR_VIT_BASE
CHECKPOINT = "google/vit-base-patch16-224"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"  # Establece el dispositivo.
logger.info(f"Dispositivo actual: {DEVICE}")

# MLflow: Configuración de la URI de seguimiento
try:
    response = requests.get(MLFLOW_URL)
    response.raise_for_status()  # Verifica si la solicitud fue exitosa.
    logger.success("Conexión a MLflow establecida correctamente.")
    os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_URL  # Configura la URI de seguimiento de MLflow.
    os.environ["MLFLOW_EXPERIMENT_NAME"] = CHECKPOINT.replace("/", "_")  # Configura el nombre del experimento de MLflow.
    os.environ["MLFLOW_TAGS"] = '{"model_family": "vit-base"}'
except Exception as e:
    logger.error(f"Error al conectar con MLflow. Tienes levantado el servidor de MLflow?")
    raise SystemExit(f"Error al conectar con MLflow: {e}")

# Prefect: Configuración de Prefect
try:
    response = requests.get(PREFECT_URL)
    response.raise_for_status()  # Verifica si la solicitud fue exitosa.
    logger.success("Conexión a Prefect establecida correctamente.")
except Exception as e:
    logger.error(f"Error al conectar con Prefect. Tienes levantado el servidor de Prefect?")
    raise SystemExit(f"Error al conectar con Prefect: {e}")

2025-06-19 11:39:30.638 | INFO     | __main__:<module>:9 - Dispositivo actual: cuda
2025-06-19 11:39:32.690 | SUCCESS  | __main__:<module>:15 - Conexión a MLflow establecida correctamente.
2025-06-19 11:39:34.731 | SUCCESS  | __main__:<module>:27 - Conexión a Prefect establecida correctamente.


<div align="center">✨Datos del proyecto:✨</div>

<p></p>

<div align="center">

| Subtitulo       | *Fine-tuning* del modelo vit-base sobre el conjunto EuroSAT-RGB                                                                       |
| --------------- | -------------------------------------------------------------------------------------------------------------------------------------- |
| **Descrpción**  | <small>Análisis exploratorio del proceso de *fine-tuning* del vit-base sobre el EuroSAT<br/> - *Tarea:* `Clasificacion`<br/>- *Modelo*: `ViT-Base`<br/> - *Dataset*: `EuroSAT` </small>|

</div>

## Tabla de contenidos
0. [Pasos previos](#pasos-previos)
1. [Introducción](#introduccion)
2. [Entrenamiento del modelo](#entrenamiento)
3. [Resultados](#resultados)

## 0. Pasos previos <a id="pasos-previos"></a>

Ejecuta desde la raíz del proyecto para descargar el dataset EuroSAT:

```bash
python -m vision_transformer.flows.cli prepare-dataset-flow huggingface
```

## 1. Introducción <a id="introduccion"></a>

En el trabajo final correspondiente al curso Visión por Computadora III se aborda un problema de clasificación de imágenes satelitales. El objetivo principal es comparar el desempeño de distintos modelos basados en la arquitectura Vision Transformer (ViT) y contrastarlos con al menos un modelo clásico basado en redes convolucionales. Esta comparación permite poner en práctica el fine-tuning de modelos preentrenados y, al mismo tiempo, analizar el comportamiento de distintas arquitecturas en un campo muy interesante como el de imágenes satelitales.

La propuesta busca enfocarse en un problema con objetivos claros y bien delimitados, que permita explorar tanto aspectos técnicos como conceptos actuales en el área de visión por computadora. En este caso, el interés está centrado en evaluar si las arquitecturas basadas en transformers presentan ventajas frente a los modelos convolucionales tradicionales, como se sugiere en algunos estudios recientes. Por ejemplo, el trabajo [Onboard Satellite Image Classification for Earth Observation: A Comparative Study of ViT Models](https://www.arxiv.org/pdf/2409.03901) reporta resultados positivos al aplicar ViT sobre imágenes satelitales. Por otro lado, el estudio realizado en el marco del curso CS231n de la Universidad de Stanford, [Vision Transformers for Robust Analysis of Satellite Imagery](https://cs231n.stanford.edu/2024/papers/vision-transformers-for-robust-analysis-of-satellite-imagery.pdf), presenta una visión más crítica al respecto, señalando limitaciones cuando se trabaja con datos fuera de distribución.

Cabe aclarar que este trabajo tiene un carácter exploratorio y se desarrolla con fines académicos. Si bien se toma como referencia los documentos, el propósito principal es aplicar los contenidos del curso en un caso concreto, más que validar o refutar resultados científicos previos.

Para el desarrollo se utiliza el conjunto de datos [EuroSAT](https://github.com/phelber/EuroSAT?tab=readme-ov-file), basado en imágenes del satélite Sentinel-2, perteneciente al programa Copernicus. Este dataset contiene más de 27.000 imágenes geo-referenciadas distribuidas en 10 clases, correspondientes a distintas categorías de uso del suelo. Las imágenes fueron recolectadas en 2015, por lo que son anteriores al surgimiento y la adopción generalizada de transformers en tareas de visión por computadora, lo cual agrega un marco interesante a la comparación propuesta.

Este primer notebook se enfoca en una exploración inicial del conjunto de datos, con el objetivo de comprender las características de las imágenes, la distribución de clases y otros aspectos relevantes para el preprocesamiento y el entrenamiento de los modelos.

## 2. Entrenamiento del modelo <a id="entrenamiento"></a>

In [3]:
dataset = load_huggingface_dataset()

2025-06-19 11:39:34.776 | INFO     | vision_transformer.dataset:load_huggingface_dataset:441 - Cargando el dataset procesado...
2025-06-19 11:39:35.249 | INFO     | vision_transformer.dataset:load_huggingface_dataset:453 - El dataset contiene múltiples conjuntos (train, test, val). Cargando todos...


Resolving data files:   0%|          | 0/24300 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/2700 [00:00<?, ?it/s]

In [4]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 24300
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 2700
    })
})


In [5]:
id2label = {id: label for id, label in enumerate(dataset["train"].features["label"].names)}
label2id = {label: id for id, label in id2label.items()}

print("Cantidad de clases:", len(id2label), "\n")
for k, v in id2label.items():
    print(f"- id {k}: {v}")

Cantidad de clases: 10 

- id 0: AnnualCrop
- id 1: Forest
- id 2: HerbaceousVegetation
- id 3: Highway
- id 4: Industrial
- id 5: Pasture
- id 6: PermanentCrop
- id 7: Residential
- id 8: River
- id 9: SeaLake


En este notebook se trabaja con ViT Base, el primer modelo presentado por Google en el paper [An Image is Worth 16x16 Words](https://arxiv.org/pdf/2010.11929). Este modelo marcó el inicio del uso de transformers en visión por computadora, y se toma como punto de partida para establecer una primera línea de base que luego se podrá comparar con variantes como CvT o Swin Transformer.

A continuación se incluye un esquema visual del modelo para facilitar su interpretación:

<div align="center"><img src="../resources/../resources/vit.png" width="600" alt="Figura 1: Vit Base"></div>
<div align="center"><small><em>Figura 1: Vit Base</em></small></div>

La implementación del modelo se realiza utilizando la librería [transformers](https://huggingface.co/docs/transformers/en/index) de Hugging Face. En este caso, se utiliza la versión [google/vit-base-patch16-224](https://huggingface.co/google/vit-base-patch16-224), una de las variantes originales publicadas por el equipo de Google Research.

Esta versión del modelo fue preentrenada con imágenes de tamaño 224×224, lo cual se alinea con el preprocesamiento realizado en este proyecto. Cuenta con aproximadamente 86 millones de parámetros, lo que representa un buen equilibrio entre complejidad y eficiencia para trabajar en un entorno de prueba y ajuste como el planteado en este trabajo.

In [6]:
processor = ViTImageProcessor.from_pretrained(CHECKPOINT)
processor

ViTImageProcessor {
  "do_convert_rgb": null,
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

In [7]:
model = ViTForImageClassification.from_pretrained(
    CHECKPOINT,
    num_labels=10,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([10, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
transforms = VitBaseTransforms(image_processor=processor)
print(transforms.transforms_to_string())

Transformaciones de entrenamiento:
  - RandomApply(
    p=0.8
    RandomRotation(degrees=[-15.0, 15.0], interpolation=nearest, expand=False, fill=0)
)
  - RandomApply(
    p=0.8
    Resize(size=(72, 72), interpolation=bicubic, max_size=None, antialias=True)
    RandomCrop(size=(64, 64), padding=0)
)
  - RandomHorizontalFlip(p=0.5)
  - RandomApply(
    p=0.8
    ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=(0.8, 1.2), hue=(-0.1, 0.1))
)
  - Resize(size=(224, 224), interpolation=bicubic, max_size=None, antialias=True)
  - ToTensor()
  - Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])

Transformaciones de validacion:
  - Resize(size=(224, 224), interpolation=bicubic, max_size=None, antialias=True)
  - ToTensor()
  - Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])


In [9]:
# Aplicamos las transformaciones
def train_transform_wrapper(example):
    return transforms(example, train=True)


def test_transform_wrapper(example):
    return transforms(example, train=False)


encoded_ds = {
    "train": dataset["train"].with_transform(train_transform_wrapper),
    "test": dataset["test"].with_transform(test_transform_wrapper),
}

# Mostramos un ejemplo de la estructura.
print(encoded_ds["train"][0])
print(encoded_ds["test"][0]["pixel_values"].shape)

{'pixel_values': tensor([[[ 0.3255,  0.3255,  0.3255,  ..., -0.0196, -0.0118, -0.0039],
         [ 0.3255,  0.3255,  0.3255,  ..., -0.0275, -0.0196, -0.0118],
         [ 0.3255,  0.3255,  0.3255,  ..., -0.0353, -0.0275, -0.0196],
         ...,
         [-0.2471, -0.1765,  0.0039,  ..., -0.9686, -0.9686, -0.9686],
         [-0.2549, -0.1843, -0.0039,  ..., -0.9686, -0.9686, -0.9686],
         [-0.2627, -0.1922, -0.0118,  ..., -0.9686, -0.9686, -0.9686]],

        [[ 0.0745,  0.0745,  0.0745,  ..., -0.0980, -0.0902, -0.0824],
         [ 0.0745,  0.0745,  0.0745,  ..., -0.0980, -0.0902, -0.0824],
         [ 0.0745,  0.0745,  0.0745,  ..., -0.1059, -0.0980, -0.0902],
         ...,
         [-0.4196, -0.3647, -0.2157,  ..., -0.9686, -0.9686, -0.9686],
         [-0.4196, -0.3647, -0.2235,  ..., -0.9686, -0.9686, -0.9686],
         [-0.4196, -0.3647, -0.2235,  ..., -0.9686, -0.9686, -0.9686]],

        [[ 0.0745,  0.0745,  0.0745,  ..., -0.0588, -0.0510, -0.0431],
         [ 0.0745,  0.0745, 

In [10]:
clf_metrics = evaluate.combine([
    MulticlassAccuracy(),  # Precisión multicategoría.
    evaluate.load("f1"), # Puede ser "micro", "macro" o "weighted". En este caso, "weighted" toma en cuenta el desbalanceo leve de clases.
    evaluate.load("precision"), # "marco" para clases desbalanceadas.
    evaluate.load("recall")
])

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return clf_metrics.compute(predictions=predictions, references=labels, average="weighted")

In [11]:
training_args = TrainingArguments(
    output_dir=MODELS_DIR / MODEL_FOLDER,  
    overwrite_output_dir=True,  
    eval_strategy="epoch",  
    per_device_train_batch_size=64, 
    per_device_eval_batch_size=64, 
    eval_accumulation_steps=1, 
    learning_rate=5e-5,  
    num_train_epochs=10,  
    warmup_ratio=0.1,   
    save_strategy="best", 
    save_total_limit=1, 
    logging_strategy="epoch",  
    seed=RANDOM_SEED,  
    remove_unused_columns=False,  
    load_best_model_at_end=True,  
    metric_for_best_model="accuracy",
    # dataloader_num_workers=4,
    report_to=[],
)

data_collator = DefaultDataCollator()

callback_list = [EarlyStoppingCallback(early_stopping_patience=2), MLflowCallback()]

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=encoded_ds["train"],
    eval_dataset=encoded_ds["test"],
    processing_class=processor,
    compute_metrics=compute_metrics,
    callbacks=callback_list
)

In [12]:
with mlflow.start_run():
    mlflow.log_param("transforms", transforms.transforms_to_string())
    mlflow.log_param("dataset_name", DATASET_NAME)
    mlflow.log_param("dataset_version", DATASET_VERSION)

    logger.info("Iniciando entrenamiento del modelo...")
    trainer.train()
    logger.info("Entrenamiento finalizado. Guardando el modelo...")

history = pd.DataFrame(trainer.state.log_history)
history.to_csv(MODELS_DIR / MODEL_FOLDER / HISTORY_FILENAME, index=False)
history

2025-06-19 11:39:49.347 | INFO     | __main__:<module>:6 - Iniciando entrenamiento del modelo...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.617400,0.094061,0.969630,0.969578,0.970340,0.969630
2,0.097000,0.053188,0.982963,0.982919,0.983077,0.982963
3,0.060400,0.074190,0.975185,0.975254,0.976301,0.975185
4,0.044300,0.051591,0.984815,0.984842,0.985151,0.984815
5,0.030300,0.054157,0.983333,0.983366,0.983610,0.983333
6,0.022300,0.044507,0.986667,0.986658,0.986733,0.986667
7,0.015100,0.049564,0.983333,0.983345,0.983473,0.983333
8,0.009300,0.045002,0.987037,0.987038,0.987105,0.987037
9,0.006600,0.037923,0.991111,0.991110,0.991165,0.991111
10,0.005900,0.040996,0.987778,0.987808,0.987887,0.987778


2025-06-19 12:11:13.352 | INFO     | __main__:<module>:8 - Entrenamiento finalizado. Guardando el modelo...
🏃 View run exultant-calf-483 at: http://localhost:8080/#/experiments/8/runs/37b486f2e5c44f41b40e44b36d9a48b4
🧪 View experiment at: http://localhost:8080/#/experiments/8


,loss,grad_norm,learning_rate,epoch,step,eval_loss,eval_accuracy,eval_f1,eval_precision,eval_recall,eval_runtime,eval_samples_per_second,eval_steps_per_second,train_runtime,train_samples_per_second,train_steps_per_second,total_flos,train_loss
0,0.6174,4.119771,4.986842e-05,1.0,380,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,1.0,380,0.094061,0.969630,0.969578,0.970340,0.969630,27.8954,96.790,1.541,NaN,NaN,NaN,NaN,NaN
2,0.0970,2.167001,4.445906e-05,2.0,760,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,2.0,760,0.053188,0.982963,0.982919,0.983077,0.982963,7.5697,356.685,5.681,NaN,NaN,NaN,NaN,NaN
4,0.0604,4.369373,3.890351e-05,3.0,1140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,3.0,1140,0.074190,0.975185,0.975254,0.976301,0.975185,7.6981,350.737,5.586,NaN,NaN,NaN,NaN,NaN
6,0.0443,4.043529,3.334795e-05,4.0,1520,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,4.0,1520,0.051591,0.984815,0.984842,0.985151,0.984815,7.5642,356.947,5.685,NaN,NaN,NaN,NaN,NaN
8,0.0303,0.190948,2.779240e-05,5.0,1900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,5.0,1900,0.054157,0.983333,0.983366,0.983610,0.983333,7.6722,351.918,5.605,NaN,NaN,NaN,NaN,NaN


In [19]:
logger.info("Evaluando el modelo...")
metrics = trainer.evaluate()
logger.info("Evaluación finalizada. Métricas:")

metrics_df = pd.DataFrame(metrics, index=[0])
metrics_df.to_csv(MODELS_DIR / MODEL_FOLDER / METRICS_FILENAME, index=False)
metrics_df

2025-06-19 12:19:29.686 | INFO     | __main__:<module>:1 - Evaluando el modelo...


2025-06-19 12:19:40.156 | INFO     | __main__:<module>:3 - Evaluación finalizada. Métricas:


,eval_loss,eval_accuracy,eval_f1,eval_precision,eval_recall,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch
0,0.040996,0.987778,0.987808,0.987887,0.987778,7.9522,339.528,5.407,10.0


In [14]:
# Guardamos las predicciones del modelo en el conjunto de test
predictions_output = trainer.predict(encoded_ds["test"])

# Probabilidades
y_probs = predictions_output.predictions

# Predicciones finales (argmax)
y_pred = np.argmax(y_probs, axis=1)

# Etiquetas reales
y_true = predictions_output.label_ids

results_df = pd.DataFrame(
    {
        "y_true": [id2label[i] for i in y_true],
        "y_pred": [id2label[i] for i in y_pred],
    }
)
results_df.to_csv(MODEL_FOLDER / PREDICTIONS_FILENAME, index=False)

## 3. Resultados <a id="resultados"></a>

In [15]:
history = pd.read_csv(MODEL_FOLDER / HISTORY_FILENAME)
results_df = pd.read_csv(MODEL_FOLDER / PREDICTIONS_FILENAME)
y_true = results_df["y_true"].values
y_pred = results_df["y_pred"].values

In [16]:
plot_confusion_matrix(
    y_true=y_true,
    y_pred=y_pred,
    labels=sorted(set(y_true)),
    filename=CHECKPOINT.replace("/", "-") + "_confusion_matrix",
    dirpath=FIGURES_DIR / MODEL_NAME,
    show_as_percentaje=True
)

In [17]:
# Filtramos solo las filas que tienen datos útiles
filtered_history = history.copy()
filtered_history = filtered_history[filtered_history["epoch"].notna()]

# Plot de pérdidas (loss)
plot_metric(
    filtered_history,
    x_col="epoch",
    y_cols=["loss", "eval_loss"],
    y_labels=["Pérdida de entrenamiento", "Pérdida de evaluación"],
    title="Pérdida por época",
    filename=CHECKPOINT.replace("/", "-") + "_loss_plot",
    dirpath=FIGURES_DIR / MODEL_NAME,
)

# Plot de accuracy
if "eval_accuracy" in filtered_history.columns:
    plot_metric(
        filtered_history[filtered_history["eval_accuracy"].notna()],
        x_col="epoch",
        y_cols=["eval_accuracy"],
        y_labels=["Accuracy de evaluación"],
        title="Accuracy por época",
        filename=CHECKPOINT.replace("/", "-") + "_accuracy_plot",
        dirpath=FIGURES_DIR / MODEL_NAME,
    )